In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import numpy as np 
import nibabel as nib 
from glob import glob 
import matplotlib.pyplot as plt 

from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
import argparse
import numpy as np 
import pandas as pd 
import os 
from numpy.testing import assert_array_almost_equal

import numpy as np
import sklearn.datasets, sklearn.decomposition
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as sch
from scipy.cluster.hierarchy import cut_tree, fcluster, cophenet
from scipy.spatial.distance import pdist 
from scipy.stats import spearmanr
import nibabel as nib
from scipy.cluster.hierarchy import inconsistent


def get_pcs_v2(X_train, nComp=None): 
    # https://stats.stackexchange.com/questions/229092/how-to-reverse-pca-and-reconstruct-original-variables-from-several-principal-com
    
    #print("shape: {}".format(X_train.shape))
    
    # we need (TRs x Voxels)
    assert(X_train.shape[0] == 180)
    
    
    # create pca object
    # nComp determines how many PCs we want ex. 10 
    pca = sklearn.decomposition.PCA(n_components=nComp)

    # X_train_pca is the nComp PCs ex. 10 
    #  X_train_pca.shape == (180, 5)
    X_train_pca = pca.fit_transform(X_train)
    assert(X_train_pca.shape == (180,nComp) )
    
    
    # X_projected is the PC projected back into signal space 
    X_projected = pca.inverse_transform(X_train_pca)
    assert(X_projected.shape == X_train.shape)
    
    exp_var = pca.explained_variance_ratio_
    assert(exp_var.shape == (nComp,))
    
    
    loss = np.sum((X_train - X_projected) ** 2, axis=1).mean()

    # returns
    #   PCs
    #   the PCs projected to singnal space (same dimensions as input data)
    #   explained variance for each PC 
    #   loss    
    return X_train_pca, X_projected, exp_var, loss


work_dir="/data/NIMH_scratch/kleinrl/ds003216-download/derivatives/sub-02/VASO_fun2_afni"
data_dir=work_dir+"/data"

out_dir=work_dir+"/out"
roi_dir=work_dir+"/rois"
timeseries_maindir=work_dir+"/timeseries"
plot_dir=work_dir+"/plots_despike"

img_path = "/data/NIMH_scratch/kleinrl/gdown/parc_hcp_kenshu_uthr.nii.gz"
img = nib.load(img_path)
img_data = img.get_fdata()
            

sess = glob(timeseries_maindir+"/VASO_grandmean_WITHOUT-ses-13_spc_despike")
sess+= glob(timeseries_maindir+"/sub*_spc_despike")
#sess = sess[0:3]

roi="FEF"
#types = ["ward"] #, "centroid", "median", "weighted", "average", "complete", "single" ]
types = ["ward", "centroid", "median", "weighted", "average", "complete", "single" ]


cophenet_coefs = []
cophenet_coefs.append(['ses', 'type', 'i', 'corr_coef_sp', 'corr_coef_pe'])

ses = sess[0]
i   = 30
type= "PCR"

plot_file           = plot_dir+'/clust_'+type+"_"+str(i)+"_"+ses.split('/')[-1]+"_"+roi+".jpeg"    
plot_file_hist      = plot_dir+'/hist_'+type+"_"+str(i)+"_"+ses.split('/')[-1]+"_"+roi+".jpeg"    
plot_file_voxs      = plot_dir+'/voxs_'+type+"_"+str(i)+"_"+ses.split('/')[-1]+"_"+roi+".jpeg"    
plot_nifti_clusters = data_dir+'/clustVoxs2_'+type+"_"+str(i)+"_"+ses.split('/')[-1]+"_"+roi+".nii.gz"  


#ts_FEF_2D=glob(ses+"/*"+roi+"*.2D")
ts_FEF_2D=glob(ses+"/*"+roi+"*ijk.2D")
ts_FEF_1D=glob(ses+"/*"+roi+"*.1D")

ts_FEF_2D.sort()
ts_FEF_1D.sort()


FEF     = [ np.loadtxt(x) for x in ts_FEF_2D ] 
FEF_ind = np.concatenate([len(x)*[y+1] for x,y in zip(FEF, range(0,len(FEF))) ])
FEF_cat = np.concatenate(FEF)

FEF_ijk = FEF_cat[:,:3]
FEF_cat = FEF_cat[:,3:]

X_train_pca, X_projected, exp_var, loss       = get_pcs_v2(FEF_cat.T, nComp=i)

X = pd.DataFrame(FEF_cat.T)


pca = PCA( n_components=10)
X_reduced_scale = pca.fit_transform(scale(X))
X_reduced = pca.fit_transform(X)

assert_array_almost_equal(X_reduced_scale, X_reduced)



#define cross validation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

regr = LinearRegression()
mse = []

# Calculate MSE with only the intercept
score = -1*model_selection.cross_val_score(regr,
           np.ones((len(X_reduced),1)), y, cv=cv,
           scoring='neg_mean_squared_error').mean()    
mse.append(score)

# Calculate MSE using cross-validation, adding one component at a time
for i in np.arange(1, 6):
    score = -1*model_selection.cross_val_score(regr,
               X_reduced[:,:i], y, cv=cv, scoring='neg_mean_squared_error').mean()
    mse.append(score)
    
# Plot cross-validation results    
plt.plot(mse)
plt.xlabel('Number of Principal Components')
plt.ylabel('MSE')
plt.title('hp')